# Parameters

In [1]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
METRICS_MONTH_TEXT = "2019-04"
MEDIAWIKI_HISTORY_SNAPSHOT = "2019-04"

# Imports

In [2]:
import datetime as dt
from functools import reduce
import io
from pathlib import Path
import re
import time

from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import wmfdata as wmf
from wmfdata import hive
from wmfdata.utils import mediawiki_dt, print_err, pd_display_all

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    "metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics

In [4]:
queries = {
    "active_editors": {
        "file": "queries/active_editors.sql",
        "engine": "hive"
    },
    "edits": {
        "file": "queries/edits.hql",
        "engine": "hive"
    },
    "new_editor_retention": {
        "file": "queries/new_editor_retention.hql",
         "engine": "hive"
    },
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_edits_editors": {
        "file": "queries/mobile-heavy_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_new_editor_retention": {
        "file": "queries/mobile-heavy_new_editor_retention.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running edits on hive...
Running new_editor_retention on hive...
Running mobile-heavy_new_editor_retention on hive...
Running active_editors on hive...
Running mobile-heavy_edits_editors on hive...
Running global_south_edits_editors on hive...


# Content metrics via API

In [5]:
NEW_PAGES_API = (
    "https://wikimedia.org/api/rest_v1/metrics/edited-pages/new/" +
    "{project}/all-editor-types/{page_type}/monthly/{start}/{end}"
)

headers = {
    "User-Agent": "https://github.com/wikimedia-research/Editing-movement-metrics (bot)"
}

# Create container for results
api_results = []

def get_new_pages(
    project="all-projects",
    page_type="content",
    start=date_params["api_metrics_month_first_day"],
    end=date_params["api_metrics_month_day_after"]
):
    url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )
    
    r = requests.get(url, headers=headers)
    data = r.json()["items"][0]["results"]
    frame = pd.DataFrame(data)
    frame["timestamp"] = pd.to_datetime(frame["timestamp"])
    frame = frame.rename(columns={"timestamp": "month"})
    
    return frame

## Total

In [6]:
total_new = get_new_pages().rename(columns={"new_pages": "net_new_content_pages"})
api_results.append(total_new)

## Wikidata

In [7]:
new_wd = get_new_pages(
    project="wikidata.org"
).rename(columns={
    "new_pages": "net_new_Wikidata_entities"
})
api_results.append(new_wd)

## Commons

In [8]:
new_commons = get_new_pages(
    project="commons.wikimedia.org"
).rename(columns={
    "new_pages": "net_new_Commons_content_pages"
})
api_results.append(new_commons)

## Wikipedias

In [9]:
# Get a list of project URLs (each one in a 1-tuple)
wp_domains = hive.run("""
select domain_name
from canonical_data.wikis
where database_group = "wikipedia"
""", fmt="raw")

# Query the API for each project and append records to a list
results = []
n = len(wp_domains)

for idx, val in enumerate(wp_domains):
    domain = val[0]
    
    if idx % 10 == 0:
        msg = "Now on the {}th project of {} ({})"
        print_err(msg.format(idx, n, domain))
        
    frame = get_new_pages(project=domain).reset_index()
    frame["project"] = domain
    records = frame.to_dict("records")
    results.extend(records)
    
    # Sleep 20 milliseconds
    time.sleep(0.02)

# Turn the big list of records into a data frame
new_per_wp = pd.DataFrame(results)

# Sum across projects to get new Wikipedia articles per month
new_wp = new_per_wp.groupby("month").agg(
    {"new_pages": "sum"}
).rename(columns={"new_pages": "net_new_Wikipedia_articles"}).reset_index()

api_results.append(new_wp)

Now on the 0th project of 304 (aa.wikipedia.org)
Now on the 10th project of 304 (arc.wikipedia.org)
Now on the 20th project of 304 (bar.wikipedia.org)
Now on the 30th project of 304 (bm.wikipedia.org)
Now on the 40th project of 304 (cdo.wikipedia.org)
Now on the 50th project of 304 (cr.wikipedia.org)
Now on the 60th project of 304 (dsb.wikipedia.org)
Now on the 70th project of 304 (et.wikipedia.org)
Now on the 80th project of 304 (frr.wikipedia.org)
Now on the 90th project of 304 (gn.wikipedia.org)
Now on the 100th project of 304 (hif.wikipedia.org)
Now on the 110th project of 304 (id.wikipedia.org)
Now on the 120th project of 304 (iu.wikipedia.org)
Now on the 130th project of 304 (kg.wikipedia.org)
Now on the 140th project of 304 (kr.wikipedia.org)
Now on the 150th project of 304 (lb.wikipedia.org)
Now on the 160th project of 304 (ltg.wikipedia.org)
Now on the 170th project of 304 (mi.wikipedia.org)
Now on the 180th project of 304 (mwl.wikipedia.org)
Now on the 190th project of 304 (n

In [10]:
# Strip timezones returned by API so our month columns merge nicely
for df in api_results:
    df["month"] = df["month"].dt.tz_localize(None)

# Combining and saving metrics

In [17]:
queries["edits"]["result"] = queries["edits"]["result"].rename({"data_edits": "wikidata_edits"}, axis=1)

In [18]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]
results.extend(api_results)

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,active_editors,global_south_active_editors,global_south_edits,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,mobile_edits,net_new_Commons_content_pages,net_new_Wikidata_entities,net_new_Wikipedia_articles,net_new_content_pages,new_active_editors,new_editor_retention,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits
month,,,,,,,,,,,,,,,,,,,,,
2018-12-01,81075.0,20359.0,2067577.0,2067577.0,4046.0,1125381.0,0.047045,488735.0,1260737.0,512338.0,947571.0,186233.0,2128146.0,14753.0,0.072074,10991655.0,65458.0,0.085478,41988394.0,546299.0,21418466.0
2019-01-01,86776.0,21367.0,2434941.0,2434941.0,4228.0,988233.0,0.043358,512674.0,1416777.0,542425.0,951976.0,200366.0,1938938.0,17108.0,0.054232,12143464.0,68813.0,0.081303,46312638.0,580719.0,23213770.0
2019-02-01,82554.0,19743.0,2241877.0,2241875.0,3901.0,966696.0,0.045479,438437.0,1254935.0,494725.0,574002.0,195100.0,1455387.0,16335.0,0.054653,10952758.0,66219.0,0.073758,44135189.0,531456.0,22667003.0
2019-03-01,87711.0,21679.0,2294512.0,2294512.0,4252.0,996968.0,0.043950,465084.0,1379133.0,510723.0,643675.0,216257.0,1587804.0,17738.0,0.073268,11748062.0,69973.0,0.063090,49434627.0,542547.0,28490299.0
2019-04-01,84358.0,19941.0,2053568.0,2052287.0,3996.0,1412904.0,0.043767,414992.0,1356454.0,584061.0,717420.0,268254.0,1782113.0,15535.0,0.068630,10955991.0,66809.0,0.055778,51252861.0,616226.0,30126989.0


In [19]:
metrics.to_csv(FILENAME, sep="\t")